In [ ]:
# Kill the  wumpus game - Orlina Machado
# the inspiration for this game is take from the hunt of the wumpus game. 
# The game consists of one cave with 20 rooms , one wumpus , two bats and two pits , you can either shoot and kill the wumpus
#or if you move into a room where he is , he might eat you and loose the game . When you shoot at bats you kill them and move 
#to avialble set of rooms , but if you dont shoot them they take you to a random room .
#when you enter the room with pitts you fall in them and you loose the game 
#The game rules are in a cave , there are 20 rooms which can be accessed by the player in set of three. when you start the game
#you are randomly allocated a room , from there you have three options to shoot ,to move or quit . If you choose to quit
#you loose the game ther it self. You can choose to shoot as well but remeber you have 5 arrows in your basket and you need to 
#use them wisely to kill wampus . 
# Also remember the three signs to navigate through a room -"you smell something terrible nearby" thers is wumpus around
#"you hear a rustling" means that there is a bat nearby
#"you feel a cold wind blowing from a nearby cavern" means theres an empty pit in the room
#Also remember if you shoot an arrow in any room whers wumpus is not there , and if hes in any adjacent or nearby rooms 
#ther is a 75% chance of waking him up then he might come to eat you . 

""" Docstring 
  This is a fictional game of killing the wumpus 
  welcome aboard have some fun playing the game player 
  Also keeps your senses vigilant , you need them 
  Good luck !!!!
  
  
  
  
"""


import random
 
class Game(object):
 
	def __init__(self):
 
		cave = {1: [2,5,8], 2: [1,3,10], 3: [2,4,12], 4: [3,5,14], 5:[1,4,6],
				6: [5,7,15], 7: [6,8,17], 8: [1,7,9], 9: [8,10,18], 10: [2,9,11], 
				11: [10,12,19], 12: [3,11,13], 13: [12,14,20], 14: [4,13,15], 15: [6,14,16], 
                16: [15,17,20], 17: [7,16,18], 18: [9,17,19], 19: [11,18,20], 20: [13,16,19]}
 
		self.cave = cave
 
		self.threats = {}
 
		self.arrows = 5
 
		self.arrow_travel_distance = 5

		self.player_pos = -1
 
 
	def safe_rooms(self):

		return list(set(self.cave.keys()).difference(self.threats.keys()))
 
 
	def initialize_board(self):
		
		for threat in ['bat','pit','wumpus']:
			pos = random.choice(self.safe_rooms())
			self.threats[pos] = threat
		self.player_pos = random.choice(self.safe_rooms())
 
 
	def bfs(self, source, dest, max_depth=5):
		
		graph = self.cave
		depth = 0
 
		def search(stack, visited, dest, depth):
			if stack == []:
				return False, -1
			if dest in stack:
				return True, depth
			visited = visited + stack
			stack = list(set([graph[v][i] for v in stack for i in range(len(graph[v]))]).difference(visited))
			depth += 1
			if depth > max_depth:
				return False, depth
			else:
				return search(stack, visited, dest, depth)
 
		return search([source], [], dest, depth)
 
 
	def warning(self, threat):
		
		if threat == 'bat':
			print("You hear a rustling.")
		elif threat == 'pit':
			print("You feel a cold wind blowing from a nearby cavern.")
		elif threat == 'wumpus':
			print("You smell something terrible nearby.")
 
 
	def get_input(self):
		
		while 1:
 
			action = input("Shoot(s) or move(m) or quit(q)? ")
			try:
				mode = str(action).lower()
				assert mode in ['s', 'm', 'q']
				break
			except (ValueError, AssertionError):
				print("This is not a valid action")
 
		if mode == 'q':							
			return 'q', 0
 
		while 1:								
			action = input("Where to? ")
			try:								
				dest = int(action)
			except ValueError:
				print("This is not even a real number.")
				continue						
 
			if mode == 'm':
				try:							
					assert dest in self.cave[self.player_pos]
					break
				except AssertionError:
					print("You cannot walk that far. Please use one of the tunnels.")
 
			elif mode == 's':
				try:							
					bfs = self.bfs(self.player_pos, dest)
					assert bfs[0] == True
					break
				except AssertionError:
					if bfs[1] == -1: 			
						print("There is no room with this number in the cave. Your arrow travels randomly.")
						dest = random.choice(self.cave.keys())
					if bfs[1] > self.arrow_travel_distance:				
						print("Arrows dont go that far.")
 
		return mode, dest
 
 
	def enter_room(self, room_no):
		
		print("Entering room {}".format(room_no))
		
		if self.threats.get(room_no) == 'bat':
			print("You encounter a bat, it transports you to a random empty room.")
			new_pos = random.choice(self.safe_rooms())
			return self.enter_room(new_pos)
		elif self.threats.get(room_no) == 'wumpus':
			print("Wumpus eats you.")
			return -1
		elif self.threats.get(room_no) == 'pit':
			print("You fall into a pit.")
			return -1
 
		for i in self.cave[room_no]:
			self.warning(self.threats.get(i))
 
		return room_no
 
 
	def shoot(self, room_no):
		
		print("Shooting an arrow into room {}".format(room_no))
		self.arrows -= 1
		threat = self.threats.get(room_no)
		if threat in ['bat', 'wumpus']:
			del self.threats[room_no]		
			if threat == 'wumpus':
				print("You killed the wumpus!!!Woopieee!!!")
				return -1
			elif threat == 'bat':
				print("You killed a bat.")
		elif threat in ['pit', None]:
			print("Arrow is lost.")
 

		if self.arrows < 1:
			print("Your arrow box is empty.")
			return -1
 
		if random.random() < 0.75:
			for room_no, threat in self.threats.items():
				if threat == 'wumpus':
					wumpus_pos = room_no					
			new_pos = random.choice(list(set(self.cave[wumpus_pos]).difference(self.threats.keys())))
			del self.threats[room_no]
			self.threats[new_pos] = 'wumpus'			
			if new_pos == self.player_pos: 
				print("Wumpus enters your room and eats you!")
				return -1
 
		return self.player_pos
 
 
	def loop(self):
 
		print("Hunt the Wumpus\n____________________________________________")
		self.initialize_board()
		self.enter_room(self.player_pos)
 
		while 1:
 
			print("You are in room {}.".format(self.player_pos), end=" ")
			print("Tunnels lead to:  {0}  {1}  {2}".format(*self.cave[self.player_pos]))
 
			action = self.get_input()		
			print()								
			if action[0] == 'm':					
				dest = action[1] 
				self.player_pos = self.enter_room(dest)
			elif action[0] == 's':				
				dest = action[1]
				self.player_pos = self.shoot(dest)
			elif action[0] == 'q':				
				self.player_pos = -1
 
			if self.player_pos == -1:			
				break																
 

		print("\nGame over!")	
 
 
if __name__ == '__main__':						
	
	WG = Game()
	WG.loop()

Hunt the Wumpus
____________________________________________
Entering room 6
You are in room 6. Tunnels lead to:  5  7  15
